In [126]:
from abc import ABC, abstractmethod

# Interfaces

Like a class, an interface can have methods, but the methods declared in interface may be abstract, meaning they have only a method _signature_, not a body/implementation. Interfaces specify what a class must do, but not how. They are class blueprints.  

Interfaces provide a consistent way of interacting with a class, and hide the specifics of any particular implementation. This way, changes in the different implementations do not affect the methods that a user/client will use to interact with the interface.

In Python, interfaces are called abstract base classes (ABCs), and their unimplemented (abstract) methods are marked with the abstractmethod decorator: `@abstractmethod`. Any class that implements the interface must be a subclass of the interface. If any interface method remains unimplemented in this subclass, the Python interpreter will raise an error.

Today we'll work with a day calendar interface. We'll make a sweeping assumption: that all events start and end exactly on an hour, represented by an integer 0-23.

In [127]:
class DayCalendar(ABC):
    """Interface for a calendar application"""

    @abstractmethod
    def add_event(self, name, start_hour, end_hour):
        """Add a new event with the given name and start/end times to this calendar."""

    @abstractmethod
    def query_events(self, start, end):
        """Returns the list of event names that happen between the given start/end times, exclusively"""


Even if this looks like a normal class, we can't make an instance of it. (If we remove the inheritance from ABC, we could!)

In [ ]:
my_cal = DayCalendar()

We anticipate our calendar will have two distinct types of users. We'll therefore write two different implementations of the interface:

* `StudentCalendar`, which allows overlapping events and is optimized for memory use
* `AdultCalendar`, which disallows overlapping events and is optimized for frequent querying

### Let's start with `StudentCalendar`.

In [129]:
class StudentCalendar(DayCalendar):
    """Class representing the calendar app for a busy, tired student."""
    
    # memory efficient because it only stores the event intervals, not an interval for each hour
    def __init__(self):
        self.events = []
        self.names = []
        
    def add_event(self, name, start_hour, end_hour):
        """Allows for overlapping events. Rejects events named `networking`, 
        `workout`, and `sleep`."""
        if name in {"networking", "workout", "sleep"}:
            raise ValueError("Not up to this event.")
            
        self.events.append((start_hour, end_hour))
        self.names.append(name)

    def overlaps(self, start1, end1, start2, end2):
        return start1 < end2 and start2 < end1

    def query_events(self, start, end):
        # almost never called (students don't plan ahead!) so can be slow
        return [self.names[index] for index in range(len(self.events))
                if self.overlaps(self.events[index][0], self.events[index][1], start, end)]

In [130]:
my_cal = StudentCalendar()

The above throws an error if we haven't yet implemented _all_ of the abstract methods in the interface. Once we're done implementing those, we can see our calendar in action:

In [ ]:
my_cal.add_event("6.009 Studying", 9, 11)
my_cal.add_event("Nap", 10, 11)
my_cal.add_event("Skype Friend", 20, 22)
my_cal.query_events(10, 12)

In [ ]:
my_cal.query_events(13, 14)

In [ ]:
my_cal.query_events(21, 23)

But what _is_ this calendar we created?

In [ ]:
print(type(my_cal))
print(isinstance(my_cal, StudentCalendar))

In [ ]:
print(isinstance(my_cal, DayCalendar))

So instances of `StudentCalendar` are instances of the interface `DayCalendar`, too! Indeed, as we saw above, any instance of `DayCalendar` must also be an instance of one of the interface's implementations.

### Now let's make `AdultCalendar`.

In [136]:
class AdultCalendar(DayCalendar):
    """Class representing the calendar app for a responsible grown up."""
    
    # not memory efficient since it stores events repeatedly, for each hour they occur
    def __init__(self):
        self.hours_to_events = {hr: set() for hr in range(24)}

    def add_event(self, name, start_hour, end_hour):
        """Raises ValueError if event is overlapping."""
        if self.query_events(start_hour, end_hour):
            raise ValueError("You have a conflict!")

        for hour in range(start_hour, end_hour):
            self.hours_to_events[hour].add(name)

    def query_events(self, start, end):
        # Called often, so needs to be more efficient
        events = set()
        for hour in range(start, end):
            events.update(self.hours_to_events[hour])
            
        return list(events)

In [137]:
my_cal = AdultCalendar()
my_cal.add_event("Do Taxes", 15, 18)
my_cal.add_event("Visit DMV", 9, 10)
my_cal.add_event("Networking", 12, 13)

In [ ]:
my_cal.add_event("Fancy Gala", 17, 20)

In [ ]:
my_cal.query_events(8, 11)

### Time to compare these!

In [140]:
# DON'T WORRY ABOUT THIS CODE! (But you might like to check it out for your lab8 demo.)
import time 

def make_events(cal):
    """Makes a bunch of non-overlapping events"""
    for start in range(24):
        if start != 15: # so that hour 15 is the quietest
            cal.add_event(f"Event{start}", start, start + 1)
    
def time_queries(cal):
    """Makes a bunch of queries and times them"""
    total_time = 0
    for _ in range(500):  
        for duration in range(24):
            for start in range(24 - duration):
                timing_start = time.time()
                cal.query_events(start, start + duration)
                total_time += (time.time() - timing_start)
    return total_time

In [141]:
mit_stud_cal = StudentCalendar()
make_events(mit_stud_cal)
print("StudentCalendar queries took", time_queries(mit_stud_cal), "seconds")

prof_cal = AdultCalendar()
make_events(prof_cal)
print("AdultCalendar queries took", time_queries(prof_cal), "seconds")

StudentCalendar queries took 2.1319901943206787 seconds
AdultCalendar queries took 0.8571815490722656 seconds


In [142]:
# DON'T WORRY ABOUT THIS CODE! (But you might like to check it out for your lab8 demo.)
# Source: good ole stack overflow -- https://stackoverflow.com/questions/449560/how-do-i-determine-the-size-of-an-object-in-python

import sys
from types import ModuleType, FunctionType
from gc import get_referents

def get_size(obj):
    """Returns an estimate of the memory/size of the object."""
    if isinstance(obj, (type, ModuleType, FunctionType)):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, (type, ModuleType, FunctionType)) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [143]:
print("StudentCalendar has size", get_size(mit_stud_cal))
print("AdultCalendar has size", get_size(prof_cal))

StudentCalendar has size 4142
AdultCalendar has size 8674


So, as we hoped, the calendar for our students is compact space-wise, but can afford to be slow on queries. On the other hand, our adults can afford to take up space on their fancy computers, but just want to know their schedules at lightening speed. All users are served by our interface!

### Using our Calendars

Now we want to know, given one of our calendars, when the best time is to take a nap. That is, let's find the hour during the day with the fewest number of events.

In [144]:
def get_quietest_hour(calendar):
        """Return any of the start hours with the fewest events."""
        return min([hour for hour in range(24)], key= lambda h: len(calendar.query_events(h, h + 1)))

In [ ]:
get_quietest_hour(mit_stud_cal)  # should be 15

In [ ]:
get_quietest_hour(prof_cal)  # should be 15

Since this function is given an instance of `DayCalendar` and only relies on the interface methods of `DayCalendar`, a good object-oriented design would be to implement it inside the interface:

In [147]:
class DayCalendar(ABC):
    """Interface for a calendar application"""

    @abstractmethod
    def add_event(self, name, start_hour, end_hour):
        """Add a new event with the given name and start/end times to this calendar."""

    @abstractmethod
    def query_events(self, start, end):
        """Returns the list of event names that happen between the given start/end times, exclusively"""

    # NEW
    def get_quietest_hour(self):
        """Return any of the start hours with the fewest events."""
        return min([hour for hour in range(24)], key= lambda h: len(self.query_events(h, h + 1)))

Since this method doesn't have the `abstractmethod` decorator, it need not be implemented in the interface implementations. (You'll need to rerun the implementation class definitions, after running this new interface definition, for the following to not throw errors.)

In [ ]:
mit_stud_cal = StudentCalendar()
make_events(mit_stud_cal)
mit_stud_cal.get_quietest_hour()

In [ ]:
prof_cal = AdultCalendar()
make_events(prof_cal)
prof_cal.get_quietest_hour()